In [1]:
import tensorflow as tf
from tensorflow import keras

In [3]:
from sklearn.model_selection import train_test_split

(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=5000, random_state=42)

In [4]:
train_set = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(len(X_train))
test_set = tf.data.Dataset.from_tensor_slices((X_test, y_test))
valid_set = tf.data.Dataset.from_tensor_slices((X_valid, y_valid))

In [8]:
from tensorflow.train import Feature, Features, Example, BytesList, Int64List

def create_example(image, label):
    image_data = tf.io.serialize_tensor(image)
    return Example(
        features=Features(
            feature={
                "image": Feature(bytes_list=BytesList(value=[image_data.numpy()])),
                "label": Feature(int64_list=Int64List(value=[label]))
            }
        )
    )

In [10]:
for image, label in valid_set.take(1):
    print(create_example(image, label))

features {
  feature {
    key: "image"
    value {
      bytes_list {
        value: "\010\004\022\010\022\002\010\034\022\002\010\034\"\220\006\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\000\002\004\000\000\223}jq\033\000\001\000\000\000\000\000\000\000\000\000\000\000\000\001\001\000\000\000\003\000\033\306;\000\000\020|(\000\002\001\000\000\000\000\000\000\000\000\000\000\000\000\001\002\000\002\000\2552\000\000\000\000\000\211\000\000\000\001\001\000\000\001\000\000\000\000\000\000\000\000\001\000\000\003\214\000\000\000\000\n\000e-\000\004\000\000\001\001\000\000\000\000\000\000\000\001\000\000\004\000\035\221\000\000\002\002\003\000+e\000\007\000\000\000\001\000\000\000\000\000\000\000\000\000\000\002\000i\213\000\000\000\000\001\000\007\233\000\000\000\000\000\000\000\

In [24]:
from contextlib import ExitStack

def to_tfrecords(name, dataset, n_shards=5):
    paths = ["{}.tfrecord-{:02d}-of-{:02d}".format(name, index, n_shards) for index in range(n_shards)]
    with ExitStack() as stack:
        writers = [stack.enter_context(tf.io.TFRecordWriter(path)) for path in paths]
        for index, (image, label) in dataset.enumerate():
            shard = index % n_shards
            example = create_example(image, label)
            writers[shard].write(example.SerializeToString())
    return paths

In [25]:
train_filepaths = to_tfrecords("./tfrecords/my_fashion_mnist.train", train_set)
valid_filepaths = to_tfrecords("./tfrecords/my_fashion_mnist.valid", valid_set)
test_filepaths = to_tfrecords("./tfrecords/my_fashion_mnist.test", test_set)

In [ ]:
def proprecessing(inputs):
    feature_descriptions={
        "image": tf.io.FixedLenFeature([], tf.string, default_value=""),
        "label": tf.io.FixedLenFeature([], tf.int64, default_value=-1)
    }
    